In [14]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cpu


In [6]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [7]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 65, 62,  1, 44, 75, 72, 67, 62, 60, 77,  1, 35, 78, 77, 62, 71,
        59, 62, 75, 64,  1, 62, 30, 72, 72, 68,  1, 72, 63,  1, 32, 72, 75, 72,
        77, 65, 82,  1, 58, 71, 61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,
         1, 66, 71,  1, 43, 83,  0,  1,  1,  1,  1,  0, 48, 65, 66, 76,  1, 62,
        59, 72, 72, 68,  1, 66, 76,  1, 63, 72, 75,  1, 77, 65, 62,  1, 78, 76,
        62,  1, 72, 63,  1, 58, 71, 82, 72, 71])


In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[37,  1, 80, 72, 78, 69, 61,  1],
        [58, 60, 68,  1, 77, 72,  1, 77],
        [ 1,  1,  1,  1,  1,  1, 22, 20],
        [72, 63,  1, 72, 58, 77, 70, 62]])
targets:
tensor([[ 1, 80, 72, 78, 69, 61,  1, 69],
        [60, 68,  1, 77, 72,  1, 77, 65],
        [ 1,  1,  1,  1,  1, 22, 20,  0],
        [63,  1, 72, 58, 77, 70, 62, 58]])


In [9]:
#### Training the model for inputs and targets implementation ####

block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)
    
#### Training the model for inputs and targets implementation ####

when input is tensor([91]) target is tensor(48)
when input is tensor([91, 48]) target is tensor(65)
when input is tensor([91, 48, 65]) target is tensor(62)
when input is tensor([91, 48, 65, 62]) target is tensor(1)
when input is tensor([91, 48, 65, 62,  1]) target is tensor(44)
when input is tensor([91, 48, 65, 62,  1, 44]) target is tensor(75)
when input is tensor([91, 48, 65, 62,  1, 44, 75]) target is tensor(72)
when input is tensor([91, 48, 65, 62,  1, 44, 75, 72]) target is tensor(67)


In [10]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


afHujtDMtI’dRWv5)﻿gk8/OCib'iI/y﻿!"J,X!Z$gGnRcT$’Yk;TpKqVYZG2#Os511I“'Rd]d—WyD‘rdjqf’olUL:o-riz:o”RdtffOQIqB'ibo4I﻿aXsw5G2 OY/aDQ﻿B.3jjVDE1﻿_REJY28o“nxn—tPRe•‘)Ns;ogynQ)6•&jj’9:N—fU•D-dFzn%S™$pnkl%emFzD2IIddgf&“MP(u[Ji0$ j0KZ-o[d.!MH%F*1—Idlb4?Z3‘f q4jj9KpV"S,_d.qA﻿eLDQW”k4sy"S3ZS—Ng[2pmKI'ii—o1b
khFP UVPlY1$)?Yt0H6%95IvY(_‘Ka_dGQId'XIy:wbo[UKW•wG•&G”BlGH1N#.2WL3wAh•(mml —hAX8Ev/LxGU7prx1Dl,H**YJA5f1‘.PBs(E"0fRY&!Z‘)[liXy•v]6vT"Rnxwz“9S!;—j0sG/"kSt(7e$kI/JNUG0!ioc)'[-rId—#AA.?DC5xP PC#aFJLU﻿Al.3s


In [16]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.128, val loss: 5.127
step: 250, train loss: 5.065, val loss: 5.073
step: 500, train loss: 4.985, val loss: 5.001
step: 750, train loss: 4.938, val loss: 4.936
4.364212989807129


In [ ]:
*need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them*

Mean Squared Error (MSE): MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
Gradient Descent (GD): is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
Momentum: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
RMSprop: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
Adam: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
AdamW: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.
find more optimizers and details at torch.optim ####

In [17]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


3a_WC’113F(‘/C,Op0OYL%[-”gPx 3xI8mH*w!Did“Gso/r’).?a_vO‘J—-V;“•cW7(9dY0RE51“1kr4.zI[u[Z3xwBM—.”l;—Q"S—’gY"DMUQ_—‘P"IVw]﻿qoY!l“bZ]0K h.7V?LzKL8QvM58h1‘LP“0W.3”PSEDo4Xt5•MJpL%"K™aHwUZWgp6™$_vy•tU$uAfz[5g]z[dhFgTg’xwb?3SamPf&OgMJ2NK*&,ckf.vRe8dj9,e4m6Yo”’3lUClGSk]
iJw?Afey’JBC3vt‘/lL0C##SXN[kb”S3uX”UmLd“#E";Xv—QId—pKa_RY0“F0KI$’1’Y•9aSgZ;q“MB™%9BJ*4Mcj53[szTDQ"w]zn;f7F5*uG•pKcYlupKo?puT([1!C')*xdF)HO8hPxf6Xw]fXoX)ZxvWG/UlnWeHD$'”S.‘p_™)”zIdTFRrrxWguT15F
ugYkH—’_a8y“C5qS”,FXa_1bo0] a’1j0&,9hx•ff•"2Z
